# Intermediate representation (IR)

$$
\newcommand{\iv}{{\mathrm{i}\nu}}
\newcommand{\wmax}{{\omega_\mathrm{max}}}
\newcommand{\dd}{{\mathrm{d}}}
$$

$$
\begin{align}
    G(\tau)&= - \int_{-\infty}^\infty\dd{\omega} K^\mathrm{L}(\tau,\omega) \rho(\omega),\\
    K^\mathrm{L}(\tau, \omega) &=  \frac{e^{-\tau\omega}}{1+e^{-\beta\omega}}.
\end{align}
$$

Singular value expansion:


$$
K^\mathrm{L}(\tau, \omega) = \sum_{l=0}^\infty U_l(\tau) S_l V_l(\omega),
$$

for $\omega \in [-\wmax, \wmax]$ and $\tau \in [0, \beta]$.

Fourier transform to Matsubara space:

$$
U_l(\iv) \equiv \int_0^\beta \dd \tau e^{\iv \tau} U_l(\tau).
$$

Parameters:

* Statistics
* $\beta$: inverse temperature
* $\wmax$: ultraviolet cutoff
* $\epsilon$: cutoff for singular values $S_l/S_0$.

In [ ]:
using Plots
using SparseIR

lambda_ = 100.0
beta = 10.0
wmax = lambda_/beta
ϵ = 1e-10

# Fermion
basis = FiniteTempBasis(Fermionic(), beta, wmax, ϵ)

## Singular values

In [ ]:
basis.s

In [ ]:
plot(basis.s, marker=:o, yaxis=:log, legend=false, xlabel="l", ylabel="S_l")

## Basis functions

In [ ]:
# U_l(τ) at τ = 0, β/2, β
basis.u([0.0, 0.5*beta, beta])

In [ ]:
# Only for the first basis functions
basis.u[1]([0.0, 0.5*beta, beta])

In [ ]:
# V_l(ω) at ω = 0, 0.5ωmax
basis.v([0.0, 0.5*wmax])

In [ ]:
taus = collect(LinRange(0, beta, 1000))
p = plot()
for l in [1, 2, 3]
    plot!(p, taus, basis.u[l](taus), label="l=$(l-1)", xlabel="τ", ylabel="U_l(τ)")
end
p

In [ ]:
omegas = collect(LinRange(-wmax, wmax, 1000))
p = plot()
for l in [1, 2, 3]
    plot!(p, taus, basis.v[l](omegas), label="l=$(l-1)", xlabel="ω", ylabel="V_l(ω)")
end
p

In [ ]:
nmax = 100

v = FermionicFreq.(2 .* collect(0:nmax-1) .+ 1)

uhat_val = basis.uhat(v)

p = plot(xlabel="ν")
for l in [1, 2, 3]
    y = l%2 == 1 ? imag.(uhat_val[l,:]) : real.(uhat_val[l,:])
    plot!(p, y, label="l=$(l-1)")
end
p

## Expanding ρ(ω) in IR

We expand $G(\tau)$ and $\rho(\omega)$ as

$$
G(\tau) = \sum_{l=0}^{L-1} G_l U_l(\tau) + \epsilon_L,
$$

$$
\hat{G}(\mathrm{i}\nu) = \sum_{l=0}^{L-1} G_l \hat{U}_l(\mathrm{i}\nu) + \hat{\epsilon}_L,
$$

where $\epsilon_L,~\hat{\epsilon}_L \approx S_L$. The expansion coefficients $G_l$ can be determined from the spectral function as 

$$
G_l = -S_l \rho_l,
$$

where

$$
\rho_l = \int_{-\omega_\mathrm{max}}^{\omega_\mathrm{max}} \mathrm{d} \omega \rho(\omega) V_l(\omega).
$$

As a simple example, we consider a particle-hole-symmetric simple spectral function 
$\rho(\omega) = \frac{1}{2} (\delta(\omega-1) + \delta(\omega+1))$ for fermions.
The expansion coefficients are given by

$$
\rho_l = \frac{1}{2}(V_l(1) + V_l(-1)).
$$

In [ ]:
rho_l = 0.5 * (basis.v(1) + basis.v(-1))
gl = - basis.s .* rho_l

ls = collect(1:length(basis.s))
p = plot(yaxis=:log, ylims=(1e-10, 1e+1), xlabel="l", ylabel="|g_l|")
plot!(p, ls, abs.(gl) .+ 1e-20, label="gl")
plot!(p, ls, abs.(basis.s), label="sl")
p

## Exercise

Compute $G_l$ for a three-peak Gaussian spectral

In [ ]:
# Three Gaussian peaks (normalized to 1)
gaussian(x, mu, sigma) = exp(-((x-mu)/sigma)^2)/(sqrt(π)*sigma)

rho(omega) = 0.2*gaussian(omega, 0.0, 0.15) + 
    0.4*gaussian(omega, 1.0, 0.8) + 0.4*gaussian(omega, -1.0, 0.8)

omegas = LinRange(-5, 5, 1000)
plot(omegas, rho.(omegas), xlabel="ω", ylabel="ρ(ω)", label="")


In [ ]:
# Hint: use QuadGk.jl